In [1]:
import cv2
import os

In [ ]:
images = []
targets = []

In [ ]:
prefix_dir_without_mask = 'dataset/without_mask/'
prefix_dir_with_mask = 'dataset/with_mask/'

In [ ]:
content = os.listdir(prefix_dir_without_mask)
for image in content:
    try:
        image_path = prefix_dir_without_mask + image
        image = cv2.imread(image_path)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image_grey, (100, 100))
        images.append(resized_image)
        targets.append(1)
    except Exception as e:
        print("Exception occured as ",e)
    

In [ ]:
len(images)

In [ ]:
content = os.listdir(prefix_dir_with_mask)
for image in content:
    try:
        image_path = prefix_dir_with_mask + image
        image = cv2.imread(image_path)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image_grey, (100, 100))
        images.append(resized_image)
        targets.append(0)
    except Exception as e:
        print("Exception occured as ",e)

In [ ]:
len(images), len(targets)

In [ ]:
images

In [ ]:
import numpy as np
images = np.array(images)/255.0

In [ ]:
images.shape[0]

In [ ]:
images = np.resize(images, (1376, 100, 100, 1))

In [ ]:
images

In [ ]:
from keras.utils import np_utils

In [ ]:
targets = np_utils.to_categorical(targets)

In [ ]:
targets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(images, targets, test_size=0.25)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:
X_train[0]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()

model.add(Conv2D

(200, (3,3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(100, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
cp = ModelCheckpoint('model-best', verbose=0, save_best_only=True)
model.fit(X_train, Y_train, epochs = 15,  callbacks=[cp], validation_split=0.2)

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
from keras.models import load_model

In [1]:
#Importing Libraries
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder

In [2]:
model = load_model('model-best')

In [3]:
face_detect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
source = cv2.VideoCapture(0)

while 1:
    not_to_use, image = source.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detect.detectMultiScale(gray, 1.5,5)
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+w, x:x+w]
        resized_face = cv2.resize(face_roi, (100, 100))
        normalized_Face = resized_face/255
        reshaped_face = np.reshape(normalized_Face, (1, 100, 100, 1))
        result = model.predict(reshaped_face)[0]
        print("Probabilities : ",result)
        if result[0] > result[1]:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) 
            cv2.putText(image, "Safe", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        

        if result[1] > result[0]:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, "Not Safe", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
    cv2.imshow('Verzeo Project', image)
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cv2.destroyAllWindows()
source.release()
    